# Census LEHD WAC Job Density

Calculating the job density of the 9 county SF Bay Area using LEHD WAC data. This notebook calculates the number of jobs per square mile for every census tract and assigns the tracts into quintiles based on job density. 

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import sys, os

## Load Data

In [2]:
# specify paths for csv and shapefile data
dirname = os.path.dirname(os.path.realpath("__file__"))
wac2015_filepath = os.path.join(dirname, "../data/wac/ca_wac_S000_JT00_2015.csv.gz")
cxwalk_filepath = os.path.join(dirname, "../data/wac/ca_xwalk.csv.gz")
tracts_filepath = os.path.join(dirname, "../data/census_tracts/tracts_2010_4326.shp")

In [3]:
# load 2002 & 2015 census wac data, plus crosswalk file
wac2015 = pd.read_csv(wac2015_filepath, sep=",", delimiter=None, header="infer", names=None, index_col=None, usecols=None, compression="gzip")
cxwalk = pd.read_csv(cxwalk_filepath, sep=",", delimiter=None, header="infer", names=None, index_col=None, usecols=None, compression="gzip", encoding="ISO-8859-1", low_memory=False)
tracts = gpd.read_file(tracts_filepath)

## Filter and Rollup 

In [4]:
# filter crosswalk table by 9 counties of SF Bay Area
cty_fips_list = [6001, 6013, 6041, 6055, 6075, 6081, 6085, 6095, 6097]
cxwalk = cxwalk[cxwalk['cty'].isin(cty_fips_list)]

In [5]:
# keep only the block and tract id columns
cxwalk = cxwalk[['tabblk2010', 'trct']]

In [6]:
# join 2015 wac files to cxwalk using fields w_geocode and tabblk2010
wac = wac2015.merge(cxwalk, how="inner", left_on="w_geocode", right_on="tabblk2010")

In [7]:
# create a new column for total number of jobs, and filter wac df
wac['total'] = wac['C000']
wac = wac[['total', 'trct']]

In [8]:
# group and aggregate data by census tract
wac = wac.groupby('trct', as_index=False).agg(np.sum)

In [9]:
# convert the tract id column to int64 so that we can perform a join
tracts['tract_id'] = tracts['GEOID'].astype(str).astype(int)

In [10]:
# join tract geometries to job counts
tracts = tracts.merge(wac, how="inner", left_on="tract_id", right_on="trct")

## calculate job density

In [11]:
# reproject tract geometries to ca state plane 3 for calculating area correctly
tracts = tracts.to_crs(epsg=2227)

# calc area in square miles for each tract (convert from sq feet to sq mile)
tracts["area_sqmi"] = (tracts.area * 3.58701e-8)

# calc the number of jobs per sq mile for each tract
tracts["density"] = tracts["total"] / tracts["area_sqmi"]

# project tract geometries back to WGS84 for ease of use with web visualization tools
tracts = tracts.to_crs(epsg=4326)

In [12]:
# summary stats for the density
tracts.describe()

,tract_id,trct,total,area_sqmi,density
count,1.580000e+03,1.580000e+03,1580.000000,1580.000000,1580.000000
mean,6.054256e+09,6.054256e+09,2360.766456,4.447725,4914.959169
std,3.790823e+07,3.790823e+07,6110.326589,24.411902,18930.437130
min,6.001400e+09,6.001400e+09,17.000000,0.021836,0.692629
25%,6.013313e+09,6.013313e+09,408.000000,0.352145,506.430721
50%,6.075026e+09,6.075026e+09,837.500000,0.612110,1342.889572
75%,6.085505e+09,6.085505e+09,1980.000000,1.321059,3310.355705
max,6.097154e+09,6.097154e+09,125749.000000,594.097209,468772.255880


In [13]:
# what's up with this outlier that has 468,772 jobs / sq mile??? 
# Turns out it's a specific census tract in downtown SF / Financial District
tracts.loc[tracts['density'] >= 460000]

,GEOID,TRACTCE10,geometry,tract_id,trct,total,area_sqmi,density
1558,6075011700,011700,POLYGON ((-122.4001490006481 37.79413400067436...,6075011700,6075011700,100627,0.214661,468772.25588


In [14]:
# we could filter tracts outside of 3 standard deviations, 
# but this ends up removing much of downtown SF and some of Downtown Oakland.
tracts[np.abs(tracts['density'] - tracts['density'].mean()) <= (3 * tracts['density'].std())]

,GEOID,TRACTCE10,geometry,tract_id,trct,total,area_sqmi,density
0,6085509201,509201,POLYGON ((-122.0695500005131 37.40839999951005...,6085509201,6085509201,2932,0.609652,4809.297616
1,6085509401,509401,POLYGON ((-122.1083099994724 37.40807999978886...,6085509401,6085509401,1457,0.257282,5663.055386
2,6085509303,509303,"POLYGON ((-122.0875290004649 37.4097109999767,...",6085509303,6085509303,318,0.221436,1436.081788
3,6085503211,503211,POLYGON ((-121.8245509999954 37.31126999978968...,6085503211,6085503211,484,0.415362,1165.247698
4,6085503504,503504,POLYGON ((-121.8281030007893 37.33550800018327...,6085503504,6085503504,728,0.432994,1681.316072
5,6085512022,512022,"POLYGON ((-121.810146000357 37.26026399991461,...",6085512022,6085512022,358,0.398941,897.375349
6,6085512017,512017,"POLYGON ((-121.8233529994362 37.2788839995422,...",6085512017,6085512017,384,0.888697,432.093177
7,6085512026,512026,POLYGON ((-121.8410289999464 37.26599499976377...,6085512026,6085512026,726,0.834886,869.579428
8,6085512020,512020,POLYGON ((-121.8351279997968 37.27791900030734...,6085512020,6085512020,577,0.406190,1420.516234
9,6085512023,512023,POLYGON ((-121.8193580007886 37.25888399947583...,6085512023,6085512023,479,0.366449,1307.139955


In [15]:
# compute the quintiles for job density
tracts['quintile'] = pd.qcut(tracts["density"], 5, labels=False)

# view the breaks for each quintile (they get listed below at the very bottom of the output)
print(pd.qcut(tracts["density"], 5))

0       (4198.004, 468772.256]
1       (4198.004, 468772.256]
2          (969.515, 1831.309]
3          (969.515, 1831.309]
4          (969.515, 1831.309]
5           (388.232, 969.515]
6           (388.232, 969.515]
7           (388.232, 969.515]
8          (969.515, 1831.309]
9          (969.515, 1831.309]
10         (969.515, 1831.309]
11        (1831.309, 4198.004]
12      (4198.004, 468772.256]
13          (388.232, 969.515]
14          (388.232, 969.515]
15          (388.232, 969.515]
16         (969.515, 1831.309]
17         (969.515, 1831.309]
18          (388.232, 969.515]
19          (388.232, 969.515]
20            (0.692, 388.232]
21            (0.692, 388.232]
22            (0.692, 388.232]
23            (0.692, 388.232]
24          (388.232, 969.515]
25            (0.692, 388.232]
26        (1831.309, 4198.004]
27         (969.515, 1831.309]
28      (4198.004, 468772.256]
29         (969.515, 1831.309]
                 ...          
1550       (969.515, 1831.309]
1551    

## Output to Shapefile

In [16]:
# columns to keep for output shp
to_keep = ['GEOID', 'total', 'area_sqmi', 'density', 'quintile', 'geometry']
tracts = tracts[to_keep]
outfile = os.path.join(dirname, 'wac_job_density.shp')
tracts.to_file(outfile)